# Proofread in eCREST

The files generated by this script will also be able to be opened in CREST original (though some information may be lost if using original CREST.py or .exe).

## Setup

Do the following two setup steps regardless of how you will be using this script. 

### 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [2]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique,nan
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import neuroglancer

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


# USING THE CREST_JSON class

## Settings definitions

Whether you are converting from neuroglancer or creating a new reconstruction, the settings_dict parameters is needed to create CREST json files with correct formatting. 
- 'save_dir' : the directory where JSON files are saved 
- 'cred' and 'db_path' : specify the path to the agglomeration database file on your local computer. 

In [3]:
settings_dict = {
    'save_dir' : '/Users/kperks/Documents/eCREST-local-files/in-progress',
    'db_path' : '/Users/kperks/Documents/eCREST-local-files/Mariela_bigquery_exports_agglo_v230111c_16_crest_proofreading_database.db',
    'max_num_base_added' : 1000,
    'cell_structures' : ['unknown','axon', 'basal dendrite', 'apical dendrite', 'dendrite', 'multiple'],
    'annotation_points' : ['exit volume', 'natural end', 'uncertain', 'pre-synaptic', 'post-synaptic']
}

### Import settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [4]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

## Proofread using eCREST



### 1. Create a crest_json object that launches a proofreading instance of neuroglancer


Initialize with either:
- (segment_id, segment_list): the main_base_id from the neuroglancer file you are converting and a list of base_segments.
- (segment_id): a "main_base_id"
- (filepath): an existing CREST json file

#### NEW reconstruction from segment ID

If you wanted to start reconstructing a new cell from a main base segment, 
you would use the following code block to launch

In [ ]:
segment_id = 474542263
crest2 = ecrest(settings_dict,segment_id = segment_id, launch_viewer=True)
# viewer_object = crest.neuroglancer_viewer()
# crest.load_to_viewer()


#### EDIT reconstruction from file

If you wanted to edit a reconstruction from an existing file, 
you would use the following code block to launch

In [ ]:
json_path = Path(settings_dict['save_dir']) / 'todo_presynaptic/sg1'#/ 'todo_postsynaptic_mg' ## 'todo_pre-synaptic/basal-dendrite' #/ 'check-duplicates' #'CC unsure'
filename = 'cell_graph_137926651__2023-06-11 19.38.24.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)
# viewer_object = crest.neuroglancer_viewer()
# crest.load_to_viewer()

with crest.viewer.config_state.txn() as s:
    s.input_event_bindings.data_view["alt+mousedown0"]="add-or-remove-seg"
    s.input_event_bindings.data_view["alt+mousedown2"]="mark-branch-in-colour"
    print(s.input_event_bindings.data_view)

In [ ]:
crest.cell_data['removed_base_segs']=set()

#### If you want to change key bindings for functions..

## check for duplicates (single cell)

In [979]:
base_segments = crest.get_base_segments_dict(Path(settings_dict['save_dir']) / 'todo_presynaptic/sg1')
df = crest.check_duplicates(base_segments)
display(df)

,self,dups,overlap-percent,number_seg_lap
74,137926651,220413578,1.0,2.0


In [973]:
# Assign the cell type then run the code cell
cell_type = 'uk'

## Do not edit
method = 'manual' # define which method you are using (manual or auto)
crest.define_ctype(cell_type,method)

### 2. SAVE YOUR WORK BEFORE CLOSING NEUROGLANCER! 

In [978]:
crest.save_cell_graph(directory_path = Path(settings_dict['save_dir']) / 'todo_presynaptic/sg1')

Saved cell 137926651 reconstruction locally at 2023-06-13 06.31.32


If you want to re-write the file you opened instead of saving with a new timestamp in the filename, run the following code cell instead of the previous one.

In [756]:
filepath = json_path / filename
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); 

Saved cell 143698547 reconstruction locally at 2023-06-09 17.59.21


#### Use the following to open a new cell in the same neuroglancer tab as is already opened

**DOES NOT WORK YET**

In [ ]:
# json_path = Path(settings_dict['save_dir']) / 'todo_post-synaptic'
# filename = 'cell_graph_302637877__2023-04-09 19.21.28.json'

# crest = ecrest(settings_dict,filepath= json_path / filename)
# crest.neuroglancer_viewer(viewer_object)
# crest.load_to_viewer()


### 3. CELL TYPING

If part of your job as a reconstructor is to identify cell types, then you can use the following blocks of code.  
First, check if it is already defined (and what the cell type was defined as).  


After you are finished defining the cell type:  
**DONT FORGET TO SAVE YOUR WORK!**. 
(step 2)

In [725]:
# Assign which method you are using (manual or auto)
method = 'manual'

## Do not edit
crest.get_ctype(method)

'sg1'

If not defined (or defined incorrectly), then define it.
> OPTIONS: mg1, mg2, mgx, lg, lf, lx, mli, gc, gran, sg

In [111]:
# Assign the cell type and which method you are using (manual or auto)
cell_type = 'lf'
method = 'manual'

## Do not edit
crest.define_ctype(cell_type,method)

## Check for DUPLICATES - single cell

Specify a folder of cells that you want to check for duplicates with the cell you are reconstructing.

The following code cell uses the function ```get_base_segments_dict``` in the crest instance to create a dictionary of all base segments for each cell within a specified directory. 

In [102]:
dirpath = Path(settings_dict['save_dir']) #/ 'todo_post-synaptic'

base_segments = crest.get_base_segments_dict(dirpath)


Load a cell that needs checking

In [ ]:
json_path = Path(settings_dict['save_dir']) / 'todo_post-synaptic' / 'check-duplicates'
filename = 'cell_graph_50844566__2023-04-21 14.47.44.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=False)

And then uses the function ```get_duplicates``` in the crest instance to check if it overlaps with any of the cells in that directory.

In [103]:
df = crest.check_duplicates(base_segments)

display(df)

,self,dups,overlap-percent,number_seg_lap


## Add found missing segments to reconstructions

manually go through each cell with missing segments, search and add them...

keep a running "todo" list of any segments that should be a new reconstruction rather than missing from current

In [25]:
missing_path = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/todo_post-synaptic/reconstructed_missing_segs-20230501.json')
with open(missing_path,'r') as fp:
    reconstructed_segs=fp.read()
    reconstructed_segs = json.loads(reconstructed_segs)


In [26]:
keys = list(reconstructed_segs.keys())
# keys
len(keys)

37

for each key in the dict, open the crest file for that cell (from nodefiles) and visualize the missing segments

In [27]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir'])
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    

In [ ]:
reconstructed_segs

In [ ]:
k = 36

crest = ecrest(settings_dict,filepath= nodefiles[keys[k]], launch_viewer=True)

print(keys[k], reconstructed_segs[keys[k]])

In [ ]:
with crest.viewer.config_state.txn() as s:
    s.input_event_bindings.data_view["alt+mousedown0"]="add-or-remove-seg"
    s.input_event_bindings.data_view["alt+mousedown2"]="mark-branch-in-colour"
    print(s.input_event_bindings.data_view)

In [81]:
crest.save_cell_graph()

Saved cell 135514741 reconstruction locally at 2023-05-09 15.30.59


In [ ]:
crest.get_ctype('manual')

In [106]:
todo = [564038367,116931244,128551991,129636736,558157595,130764619,474759791,49654133,390060758,49873267,563840123,130656616,135592261]

## Convert From Neuroglancer to eCREST

Run the following code cell to convert neuroglancer json files to eCREST json files. 

Uses "conversion_specs.json" to batch process conversion.

Conversion using "conversion_specs.json" expects:
- a folder of neuroglancer json files (with filenames standardized like in Google Drive)
- "dirname" is the folder containing neuroglancer json files to be converted
- that the "conversion_specs.json" is in the ```settings_dict['save_dir']``` key

### Batch

In [ ]:
conversion_specs_filename = "conversion_specs.json"

with open(Path(settings_dict['save_dir']) / conversion_specs_filename) as f:
    conversion_specs = json.load(f)

p = Path(conversion_specs['dirname'])

for cell_id, info in conversion_specs['cell_info'].items():
   
    f = info['filename']
    neuroglancer_layer_name = info['neuroglancer_layer_name']
    crest_layer_name = info['crest_layer_name']
  
    ## Get main_base_seg_ID from filename or from list of segment IDs
    main_base_id = f.split('_')[1] # gets the base segment ID from the name
    
    try:
        assert cell_id == main_base_id, f'cell id and filename do not match in conversion json; moving on to next cell without completing this one'
    except AssertionError as msg:
        print(msg)
        #add error message to json
        with open(settings_dict['save_dir'] / conversion_specs_filename, "r") as f:
            loaded = json.load(f)
        loaded['cell_info'][cell_id]['errors'].append(str(msg))
        with open(settings_dict['save_dir'] / conversion_specs_filename, "w") as f:
            json.dump(loaded, f, indent=4)
        continue
    
    ## Load the neuroglancer json
    print(f'you have selected cell {cell_id} to convert')
    
    with open(p / f, 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
        neuroglancer_data = json.load(myfile)

    print(f'Obtaining base_seg IDs from segmentation layer of neuroglancer json.')

    ## Obtain the list of base_segments from the neuroglancer json
    segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
    try:
        # add annotation layer
        
        base_segment_list_ng = segmentation_layer['segments']
    except TypeError as msg:
        print(msg, f': segmentation layer source is different; moving on to next cell without completing this one')
        #add error message to json
        with open(settings_dict['save_dir'] / conversion_specs_filename, "r") as f:
            loaded = json.load(f)
        loaded['cell_info'][cell_id]['errors'].append(str(msg) + f': segmentation layer source is different; moving on to next cell without completing this one')
        with open(settings_dict['save_dir'] / conversion_specs_filename, "w") as f:
            json.dump(loaded, f, indent=4)
        continue

    
    

    print(f'creating a crest_json object with no viewer for this cell')
    ## Create CREST instance with no viewer, segment_list, and segment_id
    crest = ecrest(settings_dict, segment_id = main_base_id, segment_list = base_segment_list_ng, launch_viewer=False)

    print(f'importing annotation layers from neuroglancer')
    ## Get annotations from neuroglancer -- iterate through one layer at a time to check for errors in layer names
    for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

        # get the 'layers' dictionary that has that name
        neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)

        if neuroglancer_layer != None:
            if cl_ in crest.point_types:
                # add annotation layer
                crest.import_annotations(neuroglancer_data, [nl_], [cl_])
                print(f"Imported - {nl_} - layer from neuroglancer annotations tabs for cell {crest.cell_data['metadata']['main_seg']['base']} as - {cl_} -.")
            else: 
                msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
                print(msg)
                #add error message to json
                with open(crest.save_dir / conversion_specs_filename, "r") as f:
                    loaded = json.load(f)
                loaded['cell_info'][cell_id]['errors'].append(str(msg))
                with open(crest.save_dir / conversion_specs_filename, "w") as f:
                    json.dump(loaded, f, indent=4)
        else:
            msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
            print(msg)
            #add error message to json
            with open(crest.save_dir / conversion_specs_filename, "r") as f:
                loaded = json.load(f)
            loaded['cell_info'][cell_id]['errors'].append(str(msg))
            with open(crest.save_dir / conversion_specs_filename, "w") as f:
                json.dump(loaded, f, indent=4)


    ## Save the cell_data as json
    print(f'saving cell {cell_id} with completed graph and annotations layers imported')
    crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

### Single file

Just make the "conversion_specs" file have one cell in it. The "batch" loop will still run on one cell. 

### Just annotations layer

If starting from scratch on a reconstruction is faster than converting the base_segs into a graph... but you want the annotations preserved.

In [463]:
json_path = Path(settings_dict['save_dir']) #/ 'todo_post-synaptic'
filename = 'cell_graph_42676894__2023-06-05 13.09.29.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

updating viewer status message: Current Base Segment Counts: unknown: 466, axon: 69, basal dendrite: 43, apical dendrite: 121, dendrite: 0, multiple: 36


#### from neuroglancer file

In [464]:
neuroglancer_layer_name = ['post-synaptic']
crest_layer_name = ['post-synaptic']
neuroglancer_path = '/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer_synapses/finished'
neuroglancer_path = Path(neuroglancer_path) / '42676894_nbs.json'

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)
# for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):
    # get the 'layers' dictionary that has that name
    neuroglancer_layer = next((item for item in neuroglancer_data['layers'] if item["name"] == nl_), None)

    if neuroglancer_layer != None:
        if cl_ in crest.point_types:
            # add annotation layer
            crest.import_annotations(neuroglancer_data, [nl_], [cl_])
            print(f"Imported - {nl_} - layer from neuroglancer annotations tabs for cell {crest.cell_data['metadata']['main_seg']['base']} as - {cl_} -.")
        else: 
            msg = f"CREST layer name - {cl_} - incorrect for cell {crest.cell_data['metadata']['main_seg']['base']} in conversion_json"
            print(msg)

    else:
        msg = f"no layer by the name - {nl_} - in neuroglancer json for cell {crest.cell_data['metadata']['main_seg']['base']}"
        print(msg)

crest.load_annotation_layer_points()

Imported - post-synaptic - layer from neuroglancer annotations tabs for cell 42676894 as - post-synaptic -.


In [465]:
segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
base_segment_list_ng = segmentation_layer['segments']



base_ids_added = set()
anchor_cell = crest
anchor_seg = anchor_cell.cell_data['metadata']['main_seg']['base']

segs_to_add = set(base_segment_list_ng).difference(set([a for b in anchor_cell.cell_data['base_segments'].values() for a in b]))
print(len(segs_to_add))

5


In [466]:
segs_to_add

{'!136519980', '!222394683', '!222474029', '!223617960', '!51821896'}

In [467]:
## Save the cell_data as json
print(f'saving cell {neuroglancer_path} with annotations layers imported')
crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

saving cell /Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer_synapses/finished/42676894_nbs.json with annotations layers imported
Saved cell 42676894 reconstruction locally at 2023-06-08 12.51.46


#### dictionary of just one annotation layer 

(the json file would be a list of dicts.... each dict is an annotation point)

In [ ]:
annotations_path = json_path / 'tmp' /'annotations.json'

with open(annotations_path, 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    annotate_data = json.load(myfile)
# for nl_, cl_ in zip(neuroglancer_layer_name, crest_layer_name):

# annotate_data
annotation_list = []
for v in annotate_data:


    # for v in neuroglancer_layer['annotations']:
    corrected_location = crest.get_corrected_xyz(v['point'], 'seg')

    if 'segments' not in v.keys():
        annotation_list.extend([corrected_location])
    if 'segments' in v.keys():
        annotation_list.extend([corrected_location + v['segments'][0]])

# self.cell_data['end_points'][c].extend(annotation_list)

crest.cell_data['end_points']['post-synaptic'] = annotation_list

crest.load_annotation_layer_points()

In [5]:
## Save the cell_data as json
print(f'saving cell {neuroglancer_path} with annotations layers imported')
crest.save_cell_graph() # If do not give file_path, then it will auto-generate one like CREST produces

saving cell /Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer_synapses/finished/217396339_nbs.json with annotations layers imported
Saved cell 217396339 reconstruction locally at 2023-05-07 05.10.18


### segments from an NG json into an existing CREST

In [12]:
json_path = Path(settings_dict['save_dir']) #/ 'todo_post-synaptic'
filename = 'cell_graph_476801247__2023-06-04 20.32.28.json'

crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

updating viewer status message: Current Base Segment Counts: unknown: 461, axon: 42, basal dendrite: 23, apical dendrite: 132, dendrite: 0, multiple: 0


In [216]:
neuroglancer_path = '/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/Nate_neuroglancer_synapses/finished'
neuroglancer_path = Path(neuroglancer_path) / '304356725_nbs.json'

with open(Path(neuroglancer_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
    neuroglancer_data = json.load(myfile)

In [20]:
segmentation_layer = next((item for item in neuroglancer_data['layers'] if item["source"] == 'brainmaps://10393113184:ell:roi450um_seg32fb16fb_220930'), None)
base_segment_list_ng = segmentation_layer['segments']



In [21]:
base_ids_added = set()
anchor_cell = crest
anchor_seg = anchor_cell.cell_data['metadata']['main_seg']['base']

segs_to_add = set(base_segment_list_ng).difference(set([a for b in anchor_cell.cell_data['base_segments'].values() for a in b]))
print(len(segs_to_add))

1


In [22]:
segs_to_add

{'!132313504'}

In [15]:
for base_seg in base_segment_list_ng:
    # if this segment has not already been added and it is not the anchor seg_ (should not be if not already part of cell)
    if (base_ids_added&set(base_seg)==set()) & (base_seg != anchor_seg): 
        
        displayed_segs = anchor_cell.assert_segs_in_sync(return_segs=True)
        if base_seg in displayed_segs:
            # print(f'{base_seg} already in cell, continueing')
            continue

        # print(i,base_seg)
        agglo_seg = anchor_cell.get_agglo_seg_of_base_seg(base_seg)

        constituent_base_ids = anchor_cell.get_base_segs_of_agglo_seg(agglo_seg)
        print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {crest.max_num_base_added}')


        if len(constituent_base_ids) > anchor_cell.max_num_base_added:
            base_ids = [base_seg]
            # anchor_cell.large_agglo_segs.add(agglo_seg)
            # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
        else:
            base_ids = constituent_base_ids
        
        current_segs = anchor_cell.assert_segs_in_sync(return_segs=True)

        num_base_segs_this_agglo_seg = len(base_ids)
        base_ids = [x for x in base_ids if x not in current_segs]
        num_base_segs_not_already_included = len(base_ids)
        
        # if there were segments from this agglo seg that were not in current graph, make sure you don't actually want them excluded
        if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

            base_ids = [x for x in base_ids if x not in anchor_cell.cell_data['removed_base_segs']]

            if not base_seg in base_ids:
                base_ids.append(base_seg)
        
        anchor_cell.update_base_locations(base_ids)
        anchor_cell.pr_graph.add_vertices(base_ids)

        if len(base_ids) > 1:
            edges = anchor_cell.get_edges_from_agglo_seg(agglo_seg)
            edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
            anchor_cell.pr_graph.add_edges(edges)

        join_msg = anchor_cell.add_closest_edge_to_graph(base_ids, base_seg) 
        

        # Update lists of base segments and displayed segs:
        anchor_cell.cell_data['base_segments']['unknown'].update(set(base_ids))

        with anchor_cell.viewer.txn(overwrite=True) as s:

            for bs in base_ids:
                s.layers['base_segs'].segment_colors[int(bs)] = '#ff0000' #'#d2b48c'
                s.layers['base_segs'].segments.add(int(bs))
                
        base_ids_added.update(base_ids)


        anchor_cell.update_displayed_segs() 
        anchor_cell.assert_segs_in_sync()

OperationalError: unrecognized token: "!"

In [16]:
base_seg

'!392117404'

In [11]:
anchor_cell.save_cell_graph()

Saved cell 476801247 reconstruction locally at 2023-06-04 20.32.28


## Check for duplicates in mg_network

In [545]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir']) / 'todo_pre-synaptic/sg2'
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    

In [546]:
crest = ecrest(settings_dict,launch_viewer=False)

base_segments = crest.get_base_segments_dict(dirpath)

In [547]:
df_all = pd.DataFrame()
for k,f in nodefiles.items():
    cell = ecrest(settings_dict,filepath = f,launch_viewer=False)
    df = cell.check_duplicates(base_segments)
    if not df.empty:
        df_all = pd.concat([df_all,df]) 
        
display(df_all)

,self,dups,overlap-percent,number_seg_lap
35,220399756,222673388,0.545455,30.0
32,222673388,220399756,0.576923,30.0


## visualize overlapping segments for duplicates

### get network cells segments and info

In [952]:
overlapping_cells = ['53243240','53243344'] 

In [956]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir']) / 'todo_presynaptic/sg1'

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    
# Create a base_segments dictionary of all cells

base_segments = {}
cell_type={}
for x in [n for n in nodes if n in overlapping_cells]:
    cell = ecrest(settings_dict,filepath = nodefiles[x],launch_viewer=False)
    base_segments[cell.cell_data['metadata']['main_seg']['base']] = cell.cell_data['base_segments']
    # cell_type[cell.cell_data['metadata']['main_seg']['base']] = cell.get_ctype('manual')

### Create viewer

In [957]:
viewer = neuroglancer.Viewer()
viewer.set_state({})

location=[17000,17000,1500]

with viewer.config_state.txn() as s:
    s.show_layer_panel = True ###

with viewer.txn(overwrite=True) as s:
           
    dimensions = neuroglancer.CoordinateSpace(
        scales=[16, 16, 30],# self.vx_sizes['em'],
        units='nm',
        names=['x', 'y', 'z']   )

    s.showSlices = False
    s.dimensions = dimensions
    s.position = array(location)
    s.layout = "3d"
    s.projectionScale = 30000
    

with viewer.txn(overwrite=True) as s:
    # s.projection_background_color= "#ffffff"
    s.projection_background_color= "#000000"
    

with viewer.txn(overwrite=True) as s:
    wb_open(str(viewer))

db_cursors = sqlite3_connect(settings_dict['db_path'], check_same_thread=False).cursor()

a = ', '.join(['base_address'])

db_cursors.execute(f'''SELECT {a} FROM addresses_table LIMIT 1''')

[base_seg] = db_cursors.fetchall()[0]



two_d_intensity = 0.5

with viewer.txn(overwrite=True) as s:

    s.layers['base_segs'] = neuroglancer.SegmentationLayer(source = base_seg, segments=[], segment_colors={})
    s.layers['base_segs'].ignoreNullVisibleSet = False
    s.layers['base_segs'].pick = False
    s.layers['base_segs'].selectedAlpha = two_d_intensity #For 2D

### load cells and color overlap

In [958]:
segs_1 = set([a for b in base_segments[overlapping_cells[0]].values() for a in b])
segs_2 = set([a for b in base_segments[overlapping_cells[1]].values() for a in b])

print(f'{len(segs_1.difference(segs_2))} segments in cell 1 that are not in cell 2')
print(f'{len(segs_2.difference(segs_1))} segments in cell 2 that are not in cell 1')

25 segments in cell 1 that are not in cell 2
8 segments in cell 2 that are not in cell 1


In [ ]:
segs_2.difference(segs_1)

In [521]:
overlap_seg_list = segs_1 & segs_2
print(f'{len(overlap_seg_list)} segments in both')

30 segments in both


set of cells from nodes

In [959]:

cell_color=['#33cc33','#cc33ff']
for i,n in enumerate(base_segments.keys()):
    with viewer.txn(overwrite=True) as s:
        color_structure = cell_color[i] # blue
        print(n)
        for bs in [a for b in base_segments[n].values() for a in b]:
           
            s.layers['base_segs'].segments.add(int(bs))
            s.layers['base_segs'].segment_colors[int(bs)] = color_structure # blue

color_structure='#ff0000'

with viewer.txn(overwrite=True) as s:

    for bs in list(overlap_seg_list):
        s.layers['base_segs'].segments.add(int(bs))
        s.layers['base_segs'].segment_colors[int(bs)] = color_structure # blue

53243240
53243344


In [443]:
overlap_seg_list

{'292536594'}

## Combine annotations and/or base segments across different CREST files

In [667]:
json_path = Path(settings_dict['save_dir']) #/ 'todo_pre-synaptic/sg2'

In [ ]:
['220399756','222673388']

Cell 1

In [688]:
filename = 'cell_graph_394191803__2023-06-09 12.04.04.json'

crest_1 = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

updating viewer status message: Current Base Segment Counts: unknown: 11914, axon: 0, basal dendrite: 0, apical dendrite: 0, dendrite: 0, multiple: 0


In [406]:
crest_1.save_cell_graph()

Saved cell 388682967 reconstruction locally at 2023-06-08 11.52.42


Cell 2

In [677]:
filename = 'cell_graph_393063300__2023-05-09 14.44.38.json'
# Path("/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/cell_graph_214581797__2023-04-09 10.58.57.json")

crest_2 = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

# with open(Path(ann_cell_path), 'r') as myfile: # 'p' is the dirpath and 'f' is the filename from the created 'd' dictionary
#     cell_data = json.load(myfile)

updating viewer status message: Current Base Segment Counts: unknown: 4071, axon: 0, basal dendrite: 0, apical dendrite: 0, dendrite: 0, multiple: 0


In [503]:
crest_2.save_cell_graph()

Saved cell 644005141 reconstruction locally at 2023-05-19 12.04.42


### Get missing segments from one into the other...
and adjust graph too (find missing edges and vertices... instead of making new graph, use old?)

In [689]:
segs_1 = set([a for b in crest_1.cell_data['base_segments'].values() for a in b])
segs_2 = set([a for b in crest_2.cell_data['base_segments'].values() for a in b])

print(f'{len(segs_1.difference(segs_2))} segments in cell 1 that are not in cell 2')
print(f'{len(segs_2.difference(segs_1))} segments in cell 2 that are not in cell 1')

8563 segments in cell 1 that are not in cell 2
720 segments in cell 2 that are not in cell 1


In [527]:
segs_1.difference(segs_2)

{'220399756'}

### add segments missing from one reconstruction to another

as loop... keeps track of all added (and exclude them from next iterations) because some can be in same agglo. 

In [690]:
# assign which cell you want to add to (and then keep)
anchor_cell = crest_1

# assign which segments need to be added
base_ids_all = sorted(list(segs_2.difference(segs_1)))

In [693]:
base_ids_added = set()

for base_seg in base_ids_all:
    
    if base_ids_added&set(base_seg)==set(): # if this segment has not already been added

        # print(i,base_seg)
        agglo_seg = anchor_cell.get_agglo_seg_of_base_seg(base_seg)

        constituent_base_ids = anchor_cell.get_base_segs_of_agglo_seg(agglo_seg)
        # print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {crest.max_num_base_added}')
        
        #only add from agglo those that are in the other reconstruction
        # base_ids = list(set(constituent_base_ids) & set(base_ids_all))
        base_ids = [x for x in constituent_base_ids if x in base_ids_all]


        # if len(constituent_base_ids) > anchor_cell.max_num_base_added:
        #     base_ids = [base_seg]
        #     # anchor_cell.large_agglo_segs.add(agglo_seg)
        #     # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
        # else:
        #     base_ids = constituent_base_ids
        
        current_segs = anchor_cell.assert_segs_in_sync(return_segs=True)

        num_base_segs_this_agglo_seg = len(base_ids)
        base_ids = [x for x in base_ids if x not in current_segs]
        num_base_segs_not_already_included = len(base_ids)

        if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

            base_ids = [x for x in base_ids if x not in anchor_cell.cell_data['removed_base_segs']]

            if not base_seg in base_ids:
                base_ids.append(base_seg)
        
        anchor_cell.update_base_locations(base_ids)
        anchor_cell.pr_graph.add_vertices(base_ids)

        if len(base_ids) > 1:
            edges = anchor_cell.get_edges_from_agglo_seg(agglo_seg)
            edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
            anchor_cell.pr_graph.add_edges(edges)

        join_msg = anchor_cell.add_closest_edge_to_graph(base_ids, base_seg) 
        

        # Update lists of base segments and displayed segs:
        anchor_cell.cell_data['base_segments']['unknown'].update(set(base_ids))

        with anchor_cell.viewer.txn(overwrite=True) as s:

            for bs in base_ids:
                s.layers['base_segs'].segment_colors[int(bs)] = '#ff0000' #'#d2b48c'
                s.layers['base_segs'].segments.add(int(bs))
                
        base_ids_added.update(base_ids)


        anchor_cell.update_displayed_segs() 
        anchor_cell.assert_segs_in_sync()


1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters of connected components. Connecting these clusters with nearest base segments.
1 clusters

AssertionError: 

In [687]:
len(crest_1.pr_graph.clusters(mode='weak'))

2

### Create new crest file from the union segment list...

In [332]:
new_seg_list = segs_1.union(segs_2)
segment_id = crest_1.cell_data['metadata']['main_seg']['base']

In [334]:
combo_crest = ecrest(settings_dict, segment_id = segment_id, segment_list = new_seg_list, launch_viewer=True)

Creating base segment graph for cell 561609054 Cell Reconstruction
all base locations for 462 obtained from SQL database
graph created among all_base_segs
9 clusters of connected components. Connecting these clusters with nearest base segments.
weak clusters connected
segments without a location connected
1 clusters in graph (note should/would be only 1 if loaded base ID from agglomo fresh)
Created a CREST instance for NEW Reconstruction of 561609054. No file saved yet -- save manually.
updating viewer status message: Current Base Segment Counts: unknown: 462, axon: 0, basal dendrite: 0, apical dendrite: 0, dendrite: 0, multiple: 0


Add annotations from one of the cells...

In [67]:
combo_crest.cell_data['end_points'] = crest_1.cell_data['end_points']

combo_crest.load_annotation_layer_points()

In [335]:
combo_crest.define_ctype('uk','manual')

In [336]:
combo_crest.save_cell_graph()

Saved cell 561609054 reconstruction locally at 2023-05-18 12.01.57


#### DONT FORGET TO SAVE YOUR WORK! 



## Other...

### Add vertex if missing (if can't remove a segment, sometimes this is the reason)

In [ ]:
# ('479295220')
crest.cell_data['base_segments']['unknown'].add('565168297')

In [ ]:
crest.pr_graph.vs.find("459940426")

In [ ]:
crest.pr_graph.add_vertex(name='459940426')

In [ ]:
crest.pr_graph.add_edges([(4966,323)])

### Try to add more than 1000 segments...


In [ ]:
base_seg = 215728691
agglo_seg = crest.get_agglo_seg_of_base_seg(base_seg)

In [ ]:
# First, load the original cell that is missing segments

segs_to_add_from_manual = set([a for b in crest.cell_data['base_segments'].values() for a in b])

In [ ]:
len(segs_to_add_from_manual)

In [ ]:
# then, without re-starting kernel, load the agglo cell from the missing segment

all_segs_current = set([a for b in crest.cell_data['base_segments'].values() for a in b])

In [ ]:
len(all_segs_current)

In [ ]:
# make sure you have fixed any merge errors in the agglo cell...
# then find which segments are missing from the agglo cell (because the agglo generally has more?... could do opposite)

base_ids_all = segs_to_add_from_manual - all_segs_current

In [ ]:
len(base_ids_all)

all of these segments to add are not in the agglo... so loop through as if double clicked on each independently

> FORCE IT NOT TO ADD EXTRA (because the extra could be in list and will cause graph clustering errors when tried to be added... )

In [ ]:
base_ids_all = sorted(list(base_ids_all))

In [ ]:
len(base_ids_all)

In [ ]:


for i,base_seg in enumerate(base_ids_all):

    print(i,base_seg)
    agglo_seg = crest.get_agglo_seg_of_base_seg(base_seg)

    constituent_base_ids = crest.get_base_segs_of_agglo_seg(agglo_seg)
    # print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {crest.max_num_base_added}')
    
    
    if len(constituent_base_ids) > crest.max_num_base_added:
        base_ids = [base_seg]
        self.large_agglo_segs.add(agglo_seg)
    else:
        base_ids = constituent_base_ids

    current_segs = crest.assert_segs_in_sync(return_segs=True)

    num_base_segs_this_agglo_seg = len(base_ids)
    base_ids = [x for x in base_ids if x not in current_segs]
    num_base_segs_not_already_included = len(base_ids)

    if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

        base_ids = [x for x in base_ids if x not in crest.cell_data['removed_base_segs']]

        if not base_seg in base_ids:
            base_ids.append(base_seg)

    crest.update_base_locations(base_ids)
    crest.pr_graph.add_vertices(base_ids)

    if len(base_ids) > 1:
        edges = crest.get_edges_from_agglo_seg(agglo_seg)
        edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
        crest.pr_graph.add_edges(edges)

    join_msg = crest.add_closest_edge_to_graph(base_ids, base_seg) 

    # Update lists of base segments and displayed segs:
    crest.cell_data['base_segments']['unknown'].update(set(base_ids))


    with crest.viewer.txn(overwrite=True) as s:

        for bs in base_ids:
            s.layers['base_segs'].segment_colors[int(bs)] = '#d2b48c'
            s.layers['base_segs'].segments.add(int(bs))


    crest.update_displayed_segs() 
    crest.assert_segs_in_sync()

### define cell type for a crest file

resaves as original file name (not with an updated timestamp)

In [ ]:
dirpath = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network')
filepath = dirpath / 'cell_graph_307591597__2023-04-07 12.54.44.json'
cell_type = 'lf'

### 
crest = ecrest(settings_dict, filepath = filepath, launch_viewer=False);
crest.define_ctype(cell_type,'manual')
crest.get_ctype('manual') == cell_type
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); #rewrites the original, not with a new time stamp

check cell type in neuroglancer

In [ ]:
dirpath = Path('/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network')
filepath = dirpath / 'cell_graph_213605530__2023-03-29 22.49.21.json'

crest = ecrest(settings_dict, filepath = filepath, launch_viewer=True)

In [ ]:
crest.save_cell_graph(directory_path = filepath.parent, file_name=filepath.name, save_to_cloud=False); 

### get cell types of neuroglancer reconstructions into crest json files


In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

In [ ]:
crestpath = "/Volumes/GoogleDrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"
ngpath = "/Volumes/GoogleDrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network/files_for_names"
ngfiles = [x.name for x in Path(ngpath).iterdir()]

In [ ]:
ctype_list = []
has_ctype = set()
all_cells = set()

for fname in sorted(list(Path(crestpath).iterdir())):
    if (fname.name[0]!='.') & (fname.is_file()):
        # display(fname.name)
        crest = ecrest(settings_dict, filepath = fname, launch_viewer=False);
        ngfile = list(filter(lambda x: cell.cell_data['metadata']['main_seg']['base'] in x, ngfiles))
        
        all_cells = all_cells | set({cell.cell_data['metadata']['main_seg']['base']})
        
        if len(ngfile)==1:
            ctype = ngfile[0].split('_')[3].lower()
            has_ctype = has_ctype | set({cell.cell_data['metadata']['main_seg']['base']})
        ctype_list.append(ctype)
        crest.define_ctype(ctype,'manual');
        crest.save_cell_graph(directory_path = fname.parent, file_name=fname.name, save_to_cloud=False);

In [ ]:
# make sure all crest cells have cell type definition from neuroglancer file name
all_cells-has_ctype

In [ ]:
# check cell type labels
list(unique(ctype_list))

### resave a json file with formatting for readability

In [ ]:
filepath = Path("D:\electric-fish\eCREST\CREST_settings.json")
with open(filepath, "r") as f:
    loaded = json.load(f)

with open(filepath, "w") as f:
    json.dump(loaded, f, indent=4)